# Trading Strategies Asset Pricing
### Team 63
- David Landveld
- Marc van Gils
- Paul Bejczy
- Mark de Kwaasteniet


## Trading Strategy Week 1: Weight Optimization

In [ ]:
%reset
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy
import scipy.stats
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [2]:
df_stoxx = pd.read_excel("Data_game_20211117.xlsx", index_col=0, sheet_name='PRICES')
df_stoxx = df_stoxx.pct_change()
df_stoxx = df_stoxx.iloc[1:,]
df_stoxx = df_stoxx.tail(25)


In [3]:
# Set the constraints for all portfolios that we are forming. The total sum of the weight in all the assets should be equal to 1.
# We cannot invest more than 100%, therefore, this should be equal to one.
constraint = ({'type':'eq','fun': lambda x: np.sum(x)-1}) #This is a special constraint for the scipy.optimize function. 

# Type, is the type of constraint, in this case it is an equality constraint (eq). Meaning the result should be equal to zero.
# With this function the sum of the weights minus 1 should always be equal to zero.

In [4]:
def negative_performance(weights, stock_returns = df_stoxx):
    
        # Calculate the portfolio returns using the weights given.
        portfolio_return = np.sum(weights*np.mean(stock_returns,axis=0))
    
        # Calculate the portfolio volatility. By multiplying the weights(transposed) times product of the Covariance matrix with the weights.
        portfolio_volatility = np.sqrt(np.sum(np.cov(np.transpose(stock_returns),ddof=0)*np.outer(weights,weights)))
        
        
        # Calculate the performance using the risk free rate of every formation period.
        performance = -(portfolio_return-(0.25*portfolio_volatility))
    
        # Return the Negative performance
        return performance

In [5]:
#Set the Weight bounds of the stocks
weight_bound = ((0, 1),)* len(df_stoxx.columns)

#Starting values for the optimizer
starting_values = [1/(len(df_stoxx.columns))]*len(df_stoxx.columns)

In [6]:
#Run the optimizer to find the optimal portfolio weights. 
portfolio_opt = scipy.optimize.minimize(negative_performance, starting_values, constraints=constraint, bounds=weight_bound)

#Store the portfolio weights, not rounding
weights_portfolio_opt = portfolio_opt.x

In [7]:
df_weights = pd.DataFrame(columns=['Ticker'])
df_weights['Ticker'] = df_stoxx.columns
df_weights['weight'] = weights_portfolio_opt
df_weights = df_weights.sort_values(by=['weight','Ticker'], ascending=False)
df_weights = df_weights.head(30)
df_weights.to_csv("optimal_weights.csv")

In [8]:
def portfolio_statistics(portfolio, portfolio_name):
    print('\033[1m',portfolio_name, '\033[0m')
    print('Average Daily Returns:', np.average(portfolio['return']))
    print('Average Standard Deviation of Returns:',np.std(portfolio['return']))
    print('Average Performance of Portfolio:',(np.average(portfolio['return'])- (0.25*(np.std(portfolio['return'])))))

In [9]:
df_portfolio = pd.DataFrame(columns=['return'])
for index, row in df_stoxx.iterrows():
    df_portfolio.loc[index, ['return']] = np.sum(weights_portfolio_opt*df_stoxx.loc[index])

In [10]:
portfolio_statistics(df_portfolio,'Portfolio Using 5 Weeks')

 Portfolio Using 5 Weeks 
Average Daily Returns: 0.01244283082194795
Average Standard Deviation of Returns: 0.007648746046189506
Average Performance of Portfolio: 0.010530644310400572


In [ ]:
%reset
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy
import scipy.stats

df_stoxx = pd.read_excel("Data_game_20211117.xlsx", index_col=0, sheet_name='PRICES')
df_stoxx = df_stoxx.pct_change()
df_stoxx = df_stoxx.iloc[1:,]
df_stoxx = df_stoxx.tail(25)


# Set the constraints for all portfolios that we are forming. The total sum of the weight in all the assets should be equal to 1.
# We cannot invest more than 100%, therefore, this should be equal to one.
constraint = ({'type':'eq','fun': lambda x: np.sum(x)-1}) #This is a special constraint for the scipy.optimize function. 

# Type, is the type of constraint, in this case it is an equality constraint (eq). Meaning the result should be equal to zero.
# With this function the sum of the weights minus 1 should always be equal to zero.

def negative_performance(weights, stock_returns = df_stoxx):
    
        # Calculate the portfolio returns using the weights given.
        portfolio_return = np.sum(weights*np.mean(stock_returns,axis=0))
    
        # Calculate the portfolio volatility. By multiplying the weights(transposed) times product of the Covariance matrix with the weights.
        portfolio_volatility = np.sqrt(np.sum(np.cov(np.transpose(stock_returns),ddof=0)*np.outer(weights,weights)))
        
        
        # Calculate the performance using the risk free rate of every formation period.
        performance = -(portfolio_return-(0.25*portfolio_volatility))
    
        # Return the Negative performance
        return performance

#Set the Weight bounds of the stocks
weight_bound = ((0, 1),)* len(df_stoxx.columns)

#Starting values for the optimizer
starting_values = [1/(len(df_stoxx.columns))]*len(df_stoxx.columns)

#Run the optimizer to find the optimal portfolio weights. 
portfolio_opt = scipy.optimize.minimize(negative_performance, starting_values, constraints=constraint, bounds=weight_bound)

#Store the portfolio weights, not rounding
weights_portfolio_opt = portfolio_opt.x

df_weights = pd.DataFrame(columns=['Ticker'])
df_weights['Ticker'] = df_stoxx.columns
df_weights['weight'] = weights_portfolio_opt
df_weights = df_weights.sort_values(by=['weight','Ticker'], ascending=False)
df_weights = df_weights.head(30)
df_weights.to_csv("optimal_weights.csv")

def portfolio_statistics(portfolio, portfolio_name):
    print('\033[1m',portfolio_name, '\033[0m')
    print('Average Daily Returns:', np.average(portfolio['return']))
    print('Average Standard Deviation of Returns:',np.std(portfolio['return']))
    print('Average Performance of Portfolio:',(np.average(portfolio['return'])- (0.25*(np.std(portfolio['return'])))))

df_portfolio = pd.DataFrame(columns=['return'])
for index, row in df_stoxx.iterrows():
    df_portfolio.loc[index, ['return']] = np.sum(weights_portfolio_opt*df_stoxx.loc[index])

portfolio_statistics(df_portfolio,'Portfolio Using 5 Weeks')

## Trading Strategy Week 2: Momentum + Mean Reversion

In [2]:
df_returns = pd.read_excel("Data_game_20211111.xlsx", index_col=0, sheet_name='PRICES')
df_returns = df_returns.pct_change()
df_returns = df_returns.iloc[1:,]
df_ret_st = df_returns.tail(20)
df_ret_lt = df_returns.head(198) #Exclude last week for 
df_returns #Total Dataframe is around 7 months

,BT.A-GB,CCL-GB,MKS-GB,BARC-GB,ITRK-GB,BRBY-GB,NXT-GB,KGF-GB,ITV-GB,EN-FR,...,ENR-DE,JDEP-NL,ALE-PL,WISE-GB,INPST-NL,DOCS-GB,AG1-DE,ROO-GB,ALLFG-NL,UMG-NL
2021-01-04,0.017048,-0.075473,-0.045146,-0.030877,0.029392,0.007066,-0.033860,0.009875,-0.010002,0.010995,...,0.004333,-0.049256,0.009100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,-0.002173,0.010578,0.017530,-0.001457,-0.010819,-0.012701,0.081277,0.032355,-0.010007,0.003233,...,-0.001328,-0.017933,0.021707,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-06,0.037976,0.025397,0.040401,0.078782,0.008763,0.018325,0.024951,-0.001149,0.039802,0.033988,...,0.021934,-0.021449,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-07,0.042330,0.004560,0.014439,0.008136,-0.009746,0.014806,0.002726,-0.000945,0.005251,0.015585,...,0.036098,0.009775,-0.030335,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-08,-0.022343,-0.001681,-0.020526,-0.016600,0.008472,-0.009519,0.002674,0.009272,-0.001919,-0.005301,...,0.049278,-0.023761,-0.028903,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-04,0.100187,-0.008286,0.011826,-0.048763,0.000845,0.009575,-0.002513,0.013283,0.012092,0.001138,...,0.020079,0.013019,0.019127,0.030384,0.052785,0.017875,-0.018182,-0.000699,0.008711,0.033373
2021-11-05,0.012964,0.093139,0.005779,0.003627,-0.008766,0.021157,0.003236,-0.029283,0.032691,0.011367,...,-0.023534,0.006426,0.107283,-0.039540,0.017590,0.007979,0.012346,-0.006685,-0.015199,0.004641
2021-11-08,-0.007329,-0.024451,-0.016229,0.002694,0.017766,-0.005107,-0.014156,-0.011788,-0.026258,-0.058162,...,0.049956,0.004789,-0.029971,0.010096,0.000408,-0.008605,-0.024390,0.000216,0.004326,-0.009432
2021-11-09,0.026073,-0.018954,0.021376,-0.013097,-0.000810,-0.003432,0.003081,0.008115,0.004171,-0.004177,...,0.025876,0.020651,-0.113017,0.006018,-0.025714,0.010654,0.025000,-0.001144,0.017579,0.028371


In [113]:
df_volume = pd.read_excel("Data_game_20211111.xlsx", index_col=0, sheet_name='VOLUME')
df_vol_st = df_volume.tail(21) #taking the short term volume over the last week
df_vol_st = df_vol_st.head(20)

### Momentum Strategy

In [117]:
df_mom = df_ret_lt.mean()
df_mom = df_mom.sort_values(ascending=False)
df_mom = df_mom.head(180) #buy 15 winners based on momentum?

### Mean Reversion Strategy

In [121]:
df_mr = df_ret_st.mean()
df_mr = df_mr.sort_values(ascending=False)
df_mr = df_mr.tail(60) #buy 15 losers based on mean reversion?
df_mr = df_mr.sort_values(ascending=True)
df_mr.to_csv('mean_reversion.csv')

### Including Volume

In [120]:
df_vol_mom = pd.DataFrame(df_mom, index=df_returns.columns)
df_vol_mom = df_vol_mom.sort_values(by=0,ascending=False)
df_vol_mom = df_vol_mom.head(180)
df_vol_mom = df_vol_mom.assign(volume_lt = df_volume.mean())
df_vol_mom = df_vol_mom.assign(volume_st = df_vol_st.mean())
df_vol_mom = df_vol_mom.assign(rel_volume = df_vol_mom.volume_st/df_vol_mom.volume_lt)
df_vol_mom = df_vol_mom.sort_values(by='rel_volume',ascending=True)
df_vol_mom.to_csv('momentum_final.csv')

## Strategy Week 3: Investors Sentiment - VADER analysis on Financial Times Headlines

In [93]:
df_vix = pd.read_csv("vix week 3.txt", sep=";")

In [94]:
df_vix['Date'] = pd.to_datetime(df_vix['Date'], format='%d.%m.%Y')
df_vix = df_vix.set_index('Date')

In [100]:
vix_last_month = df_vix.tail(20)
vix_second_last_month = df_vix.iloc[-40:-20,:]
vix_last_year = df_vix.tail(250)

vix_second_last_month_year = vix_second_last_month.assign(relative_vix = vix_second_last_month.Indexvalue / np.mean(vix_last_year.Indexvalue))


In [ ]:
print(np.mean(vix_second_last_month_year.relative_vix))

In [ ]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
#from webdriver_manager.chrome import ChromeDriverManager
import dateutil.parser
import time
import csv
from datetime import datetime
import io

chrome_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(executable_path=chrome_path)

driver.get('https://www.ft.com/world/europe')


In [ ]:
count = 0
headlines =[]
subtitles = []
dates = []
for x in range(151): #11 is last month #151 is till november 6 (2020)
    try:
        
        news_headlines = driver.find_elements_by_class_name('o-teaser__heading')
        second_headlines = driver.find_elements_by_class_name('o-teaser__standfirst')
        
        for headline in news_headlines:
            headlines.append(headline.text)
            
        for subtitle in second_headlines:
            subtitles.append(subtitle.text)
            
        loadMoreButton = driver.find_element_by_xpath("//a[@class='o-buttons o-buttons--secondary o-buttons-icon o-buttons-icon--arrow-right o-buttons-icon--icon-only']")
        loadMoreButton.click()
        time.sleep(3)
        
    except Exception as e:
        print(e)

In [181]:
df_subtitles_lastyear = pd.DataFrame(columns=['sub_headlines'])
df_subtitles_lastyear.sub_headlines = subtitles

In [156]:
df_headlines_lastyear = pd.DataFrame(columns=['headlines'])
df_headlines_lastyear.headlines = headlines

In [169]:
df_headlines_lastmonth = pd.DataFrame(columns=['headlines'])
df_headlines_lastmonth.headlines = headlines

In [182]:
df_subtitles_lastyear

,sub_headlines
0,Prime minister says he would ‘never’ allow ‘ve...
1,But success in supporting recovery must not br...
2,Equities in China and India decline while Turk...
3,Lira tumbles as economists fret that loose mon...
4,Candidates in next year’s presidential electio...
...,...
3742,Companies in Italy and Spain suffer a fresh fa...
3743,A long history of weak central government is c...
3744,Constitutional breach may be justified to refo...
3745,Delay seeks to give country ‘a bit of time for...


In [159]:
#Using the sentiment analyzer to analyze the sentiment of the narrative complaints in the complaints dataframe.
analyzer = SentimentIntensityAnalyzer()

#Define a quick function to get the sentiment compound score.
def get_sentiment(complaint):
    return analyzer.polarity_scores(complaint)['compound']

In [183]:
df_sent_sub_lastyear = df_subtitles_lastyear.assign(sentiment_score = df_subtitles_lastyear['sub_headlines'].apply(get_sentiment))
df_sent_sub_lastyear


,sub_headlines,sentiment_score
0,Prime minister says he would ‘never’ allow ‘ve...,0.2263
1,But success in supporting recovery must not br...,0.8720
2,Equities in China and India decline while Turk...,-0.3182
3,Lira tumbles as economists fret that loose mon...,-0.3182
4,Candidates in next year’s presidential electio...,0.0000
...,...,...
3742,Companies in Italy and Spain suffer a fresh fa...,-0.4019
3743,A long history of weak central government is c...,-0.7184
3744,Constitutional breach may be justified to refo...,0.5423
3745,Delay seeks to give country ‘a bit of time for...,-0.3182


In [175]:
np.mean(df_sent_sub.sentiment_score)

-0.06074598540145988

In [184]:
np.mean(df_sent_sub_lastyear.sentiment_score)

-0.02195025353616231

In [165]:
#Use the sentiment function to calculate the compound sentiment score of every complaint in the new dataframe.
df_sent = df_headlines_lastmonth.assign(sentiment_score = df_headlines_lastmonth['headlines'].apply(get_sentiment))
df_sent_year = df_headlines_lastyear.assign(sentiment_score = df_headlines_lastyear['headlines'].apply(get_sentiment))


In [164]:
np.mean(df_sent.sentiment_score)

-0.07032145454545449

In [167]:
np.mean(df_sent_year.sentiment_score)

-0.04533957333333326

## Strategy Week 4: Tversky Kahneman Values 

In [168]:
df_stoxx = pd.read_excel("Data_game_20211125.xlsx", index_col=0, sheet_name='PRICES')

#Keep only the prices at friday
df_stoxx = df_stoxx.iloc[::5, :]

#Calculate the cumulative returns over one week.
df_stoxx = df_stoxx.pct_change()
df_stoxx = df_stoxx.iloc[1:,]

#45 weekly returns

#Drop columns if the they contain less than 30 returns
df_stoxx = df_stoxx.dropna(thresh=30, axis=1)

In [169]:
lmda = 2.25
alpha = 0.88
delta = 0.69
gamma = 0.61

TK = []

for columns in df_stoxx.columns:
    m = df_stoxx[columns].lt(0).sum()
    n = df_stoxx[columns].count() - m
    
    weekly_ret_neg = df_stoxx[columns][df_stoxx[columns]<0].sort_values(ascending=True)
    weekly_ret_pos = df_stoxx[columns][df_stoxx[columns]>0].sort_values(ascending=True)
    
    value_ret_neg = np.float_power(abs(weekly_ret_neg.values), alpha)*lmda
    value_ret_pos = np.float_power(abs(weekly_ret_pos.values), alpha)
    
    weight_pos = []
    weight_neg = []
    
    for i in range(0,len(weekly_ret_neg)):
        p_1 = (i+1)/len(weekly_ret_neg)
        weight_1 = (np.float_power(p_1, delta))/(np.float_power((np.float_power(p_1, delta)+np.float_power((1-p_1),delta)),(1/delta)))
        
        p_2 = (i)/len(weekly_ret_neg)
        weight_2 = (np.float_power(p_2, delta))/(np.float_power((np.float_power(p_2, delta)+np.float_power((1-p_2),delta)),(1/delta)))
        weight_neg.append(weight_1-weight_2)
        
    for i in range(1,len(weekly_ret_pos)+1):
        p_1 = (n-i+1)/len(weekly_ret_pos)
        weight_1 = (np.float_power(p_1, gamma))/(np.float_power((np.float_power(p_1, gamma)+np.float_power((1-p_1),gamma)),(1/gamma)))

        p_2 = (n-i)/len(weekly_ret_pos)
        weight_2 = (np.float_power(p_2, gamma))/(np.float_power((np.float_power(p_2, gamma)+np.float_power((1-p_2),gamma)),(1/gamma)))
        
        weight_pos.append(weight_1-weight_2)
    
    TK_pos = value_ret_pos*weight_pos
    TK_neg = value_ret_neg*weight_neg
    TK_list = np.concatenate((TK_neg,TK_pos)) 
    TK_tot = np.sum(TK_neg)
    TK.append(TK_tot)
    

<ipython-input-169-b7057dbff9a3>:31: RuntimeWarning: invalid value encountered in float_power
  weight_1 = (np.float_power(p_1, gamma))/(np.float_power((np.float_power(p_1, gamma)+np.float_power((1-p_1),gamma)),(1/gamma)))
<ipython-input-169-b7057dbff9a3>:34: RuntimeWarning: invalid value encountered in float_power
  weight_2 = (np.float_power(p_2, gamma))/(np.float_power((np.float_power(p_2, gamma)+np.float_power((1-p_2),gamma)),(1/gamma)))


In [171]:
df_TK = pd.DataFrame(index=df_stoxx.columns)
df_TK = df_TK.assign(TK_value=TK)
df_TK.sort_values(by='TK_value',ascending=True)

,TK_value
SEV-FR,0.014451
SAMPO-FI,0.039623
EOAN-DE,0.042270
AV-GB,0.042886
TEL-NO,0.043288
...,...
SGRE-ES,0.205675
AMBU.B-DK,0.206793
SINCH-SE,0.222164
TMV-DE,0.229523


## Strategy Week 5: Illiquidity measure

In [4]:
df_returns = pd.read_excel("Data_game_20211125.xlsx", index_col=0, sheet_name='PRICES')
df_returns = df_returns.pct_change()
df_returns = df_returns.iloc[1:,]

In [11]:
df_turnover = pd.read_excel("Data_game_20211125.xlsx", index_col=0, sheet_name='VOLUME')
df_size = pd.read_excel("Data_game_20211125.xlsx", index_col=0, sheet_name='MV (SIZE)')

In [16]:
df_volume = df_turnover.div(df_size)
df_volume = df_volume.iloc[1:,]

In [18]:
df_illiquidity = abs(df_returns).div(df_volume)

In [20]:
df_illiquidity_last_month = df_illiquidity.tail(20)

In [28]:
df_illiquidity_last_month_mean = df_illiquidity_last_month.mean().sort_values(ascending=False)
df_illiquidity_last_month_mean.to_csv("illiquidity_df.csv")

In [27]:
df_illiquidity.mean().sort_values(ascending=False)

CDI-FR      0.365496
ALLFG-NL    0.065531
ADE-NO      0.043922
BPT-GB      0.033996
DIM-FR      0.029586
              ...   
FNTN-DE     0.001751
SEV-FR      0.001700
BAMI-IT     0.001678
UCG-IT      0.001636
LDO-IT      0.001281
Length: 600, dtype: float64